In [16]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [17]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1726693268216, experiment_id='2', last_update_time=1726693268216, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1725583395821, experiment_id='1', last_update_time=1725583395821, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1725557901398, experiment_id='0', last_update_time=1725557901398, lifecycle_stage='active', name='Default', tags={}>]

In [18]:
# client.create_experiment(name="my-cool-experiment")

In [19]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [20]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: cdfc7b99e4e947a6af41e260f4ad509b, rmse: 6.3059
run id: 62ccf330c8a54333a2d2407e6969d7da, rmse: 6.3064
run id: 1758ac2efbcc41c1b3e03eff3d4e5e79, rmse: 6.3069
run id: c70993f8ba5440af9b7430146c4e2766, rmse: 6.3094
run id: 6bf86705abc043bd9bff443aabd2d58f, rmse: 6.3100


In [21]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [22]:
run_id = "c70993f8ba5440af9b7430146c4e2766"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-xgboost")

Registered model 'nyc-taxi-xgboost' already exists. Creating a new version of this model...
Created version '7' of model 'nyc-taxi-xgboost'.


<ModelVersion: aliases=[], creation_timestamp=1727387343076, current_stage='None', description=None, last_updated_timestamp=1727387343076, name='nyc-taxi-xgboost', run_id='c70993f8ba5440af9b7430146c4e2766', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/c70993f8ba5440af9b7430146c4e2766/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>

In [23]:
model_name = "nyc-taxi-xgboost"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 7, stage: None
version: 3, stage: Staging


/tmp/ipykernel_2507/270240221.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [24]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_2507/1957332551.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1726693852230, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-09-26', last_updated_timestamp=1727387343163, name='nyc-taxi-xgboost', run_id='c70993f8ba5440af9b7430146c4e2766', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/c70993f8ba5440af9b7430146c4e2766/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [25]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1726693852230, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-09-26', last_updated_timestamp=1727387343255, name='nyc-taxi-xgboost', run_id='c70993f8ba5440af9b7430146c4e2766', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/c70993f8ba5440af9b7430146c4e2766/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [26]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [27]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [28]:
client.list_artifacts(run_id)

[]

In [29]:
# List all artifacts for the given run_id
run_id = "c70993f8ba5440af9b7430146c4e2766"
artifacts = client.list_artifacts(run_id)

# Print the list of artifacts
for artifact in artifacts:
    print(artifact.path)

In [30]:
# Try accessing the run without logging any artifacts
print("Starting run check...")
with mlflow.start_run(run_id="c70993f8ba5440af9b7430146c4e2766"):
    print("Run started successfully.")

Starting run check...
Run started successfully.


In [31]:
run_id = "f2385114aafe433e8dcc1ceaae64ad3e"
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'